# Introduksjon til RoadAI og data

For at koden under skal fungere trenger du data. Data kan lastes ned [her](https://drive.google.com/drive/folders/1_NEoph7pBfK36pVU16cwOh8r6PpkBvwV).

Det du trenger er GPSData \(og eventuelt VibrationData og AEMPdata om det skulle bli interessant\).

Legg dataen i en mappe kalt data, på samme nivå som de andre mappene. I data-mappen bør du ha GPSData med trips og tripsInfo mappene.

Har du gjort dette, bør funksjonen under returnere True!

In [ ]:
import os
os.path.isfile('./data/GPSData/trips/03-09-2022.csv')

# GPS-data

Data finnes for dager i mars, april og mai. Vi kan starte med å laste inn noe av dataen og lage noen plot.


In [ ]:
# Laster inn dataloader
import dataloader

# Bestemmer hvilken dag vi vil hente data fra
day = '04-06-2022' #MM-DD-YYYY

# Laster inn alle turer for alle maskiner som kjørte på den valgte dagen
trip = dataloader.TripsLoader(day)

# Trip er et Tripsloader-object. Mer info finner du i dataloader.py og schemas.py
# Kan printe keys, som viser oss hvilke maskiner som var i sving på den valgte dagen.
print(trip._machines.keys())


Vi kan velge en av maskinene over og se hvilken informasjon som finnes for en gitt maskin på en gitt dag.

In [ ]:
# Velger en ID
id = 22
machine = trip._machines[id]

# Kan se enkel info om maskinen
print("Machine id: ", machine.machine_id)
print("Machine type:", machine.machine_type)

# Turene til maskinen finnes under trips, som er en liste over alle turene den dagen.
# Mer info om trip finnes i schemas.py
print("Number of trips: ", len(machine.trips))

Vi kan ta en nærmere kikk på den første turen, som ligger først i listen.

In [ ]:
#Henter ut det første elementet
first_trip = machine.trips[0]

#Det finnes mye informasjon om turen
print("Trip start: ", first_trip.start_date)
print("Trip end: ", first_trip.end_date)
print("Duration (minutes): ", first_trip.duration)
print("Load of trip: ", first_trip.load)
print("Quantity (in tons): ", first_trip.quantity)
print("Distance driven (kilometers): ", first_trip.length)
print("Load latlon: ", first_trip.load_latlon)
print("Dump latlon: ", first_trip.dump_latlon)

Alle registrerte posisjoner finnes under positions. Disse kan man bruke til å estimere fart, finne flaskehalser, plotte mm.

In [ ]:
positions = first_trip.positions
print("Number of recorded positions during trip: ", len(positions))

Vi kan visualisere turen.

In [31]:
import ipyleaflet as L
from ipywidgets import Layout

m = L.Map(layout=Layout(width='60%', height='700px'), center=first_trip.load_latlon)

load_icon = L.Icon(icon_url='https://cdn-icons-png.flaticon.com/512/2716/2716797.png', icon_size=[32, 32], icon_anchor=[16,16])
load_mark = L.Marker(location=first_trip.load_latlon, icon=load_icon, rotation_angle=0, rotation_origin='22px 94px')
m.add_layer(load_mark)

dump_icon = L.Icon(icon_url='https://cdn-icons-png.flaticon.com/512/1435/1435320.png', icon_size=[32, 32], icon_anchor=[16,16])
dump_mark = L.Marker(location=first_trip.dump_latlon, icon=dump_icon, rotation_angle=0, rotation_origin='22px 94px')
m.add_layer(dump_mark)

line = L.Polyline(locations=[(pos.lat, pos.lon) for pos in first_trip.positions], color="blue" ,fill=False)
m.add_layer(line)

display(m)

Map(center=[59.951776658977145, 10.377696727405308], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [ ]:
#Videreutvikling
#Plotte en hel dag med manuelt registrerte loads and dumps
#Hente automatisk registrering fra automate_load_and_dump